# Test for consistency of arguments about matrix $M_{red}$ in the sGB ADM equations in spherical symmetry
We show here that the matrix given in the thesis in equation (3.45) from chapter Numerical setup:

\begin{align}
\underbrace{\begin{pmatrix}
1 & M_{\Phi K} & M^{rr}_{\Phi A} - \frac{\gamma^{11}}{2\gamma^{22}}  \tilde{M}_{\Phi A}  \\
0 & M_{KK} & M^{rr}_{KA} - \frac{\gamma^{11}}{2\gamma^{22}} \tilde{M}_{KA}\\
0 & M_{AK \, rr} & M_{AA \, rr}^{ \ (rr)} - \frac{\gamma^{11}}{2\gamma^{22}} \tilde{M}_{AA \, rr}\\
0 & M_{AK \, \theta\theta} & M_{AA \, \theta\theta}^{ \ (rr)} - \frac{\gamma^{11}}{2\gamma^{22}} \tilde{M}_{AA \, \theta\theta} \\
0 & M_{AK \, \phi\phi} & M_{AA \, \phi\phi}^{ \ (rr)} - \frac{\gamma^{11}}{2\gamma^{22}} \tilde{M}_{AA \, \phi\phi}
\end{pmatrix}}_{\bar{M}^D_{\text{red}}} \begin{pmatrix}\mathcal{L}_n K_{\Phi} \\ \mathcal{L}_n K \\ \mathcal{L}_n A_{rr} \end{pmatrix} = 
\begin{pmatrix}
    \mathcal{S}^{\Phi} \\
    \mathcal{S}^K \\
    \mathcal{S}^{A}_{rr} \\ 
    \mathcal{S}^{A}_{\theta\theta} \\
    \mathcal{S}^{A}_{\phi\phi} 
\end{pmatrix}
+ \frac{ A^{ij}A_{ij}}{\gamma^{22}}
\begin{pmatrix}
    \tilde{M}_{\Phi A} \\
    \tilde{M}_{K A}\\
    \tilde{M}_{AA \, rr} \\
    \tilde{M}_{AA \, \theta \theta}\\
    \tilde{M}_{AA \, \phi \phi}
\end{pmatrix},
\end{align}
where $\tilde{M}$ symbols were defined as
\begin{align}
    \tilde{M}_{\Phi A} &=  M^{\theta\theta}_{\Phi A} + \sin(\theta)^2  M^{\phi\phi}_{\Phi A},\nonumber\\
    \tilde{M}_{KA} &= M^{\theta\theta}_{KA} + \sin(\theta)^2   M^{\phi\phi}_{KA},\nonumber\\
    \tilde{M}_{AA, ii} &= M_{AA \, ii}^{ \ (\theta\theta)} + \sin(\theta)^2   M_{AA \, ii}^{ \ (\phi\phi)}, \nonumber
\end{align}
has a rows with simple linear dependency and they can be removed from the equations. Specifically, it holds that the fifth equation is a $\sin(\theta)^2$ multiple of the fourth equation and the fourth equation is a $-\frac{\gamma^{rr}A_{rr}}{2\gamma^{\theta\theta}}$ multiple of third equation. 

In [15]:
import sys
import os
import sympy as sp
import sGB_ADM_quantities as sGB

sys.path.append('nrpy')

# Import necessary NRPy+ modules:
import indexedexp as ixp
import reference_metric as rfm
import NRPy_param_funcs as par

DIM = 3

sGB.set_simplify(False) # Disable simplify in sGB. It takes too long to simplify the expressions using SymPy.
par.set_parval_from_str("reference_metric::CoordSystem","Spherical")
rfm.reference_metric()

Let the function $f(\Phi)$ be unspecified.

In [16]:
Phi = sp.Symbol('Phi')
f = sp.Function('f')(Phi)
sGB.set_f(f)

Setup the sGB ADM quantities as a general spherically symmetric data using:
$$\gamma_{ij} = \mathrm{diag}\left(\gamma_{rr}, \gamma_{\theta\theta},\gamma_{\theta\theta} \sin(\theta)^2\right),$$
$$A_{ij} = \mathrm{diag}\left(A_{rr}, -\frac{\gamma^{rr}A_{rr}}{2\gamma^{\theta\theta}}, -\frac{\gamma^{rr}A_{rr}}{2\gamma^{\theta\theta}} \sin(\theta)^2\right),$$
$$\beta^i = (\beta^r,0,0),$$
which are implemented in the sGB module.

In [17]:
sGB.setup_sGB_ADM_quantities(get_data_from='sphsym',rescaled=True)
gammaDD = sGB.gammaDD
gammaUU, _ = ixp.symm_matrix_inverter3x3(gammaDD)

Calculate $M^D_{\text{red}}$ and $S$ (expressions from equation (3.25)):

In [18]:
Mred,S = sGB.calculate_mred_S()

Calculate the reduced matrix:
- `Mredsym` (`reduced`) is the first step, i.e. matrix from equation (3.41).
    - last column of this matrix are the $\tilde{M}$ expressions that appear on the RHS of (3.46)
- `Mredsymsym` (`rereduced`) is the second step, i.e. matrix from equation (3.45). We use notation $\bar{M}^D_{\text{red}}$ in this notebook.

In [19]:
spMred = sp.Matrix(Mred)
b = sp.sin(rfm.xx[1])**2
Mredsym = []
for i in range(5):
    row = []
    for j in range(4):
        if j < 3:
            row.append(spMred[i,j])
        elif j == 3:
            row.append(spMred[i,j] + b*spMred[i,j+1])
        else:
            raise Warning('Chyba v indexech')
    Mredsym.append(row)
    
# Put into matrix for convenience.
reduced = sp.Matrix(Mredsym)
gammaUU,_ = ixp.symm_matrix_inverter3x3(gammaDD)
Mredsymsym = []
for i in range(5):
    row = []
    for j in range(3):
        if j < 2:
            row.append(reduced[i,j])
        elif j == 2:
            row.append(reduced[i,j] - gammaUU[0][0]*reduced[i,j+1]/(2*gammaUU[1][1]))
        else:
            raise Warning('Chyba v indexech')
    Mredsymsym.append(row)

# Put into matrix for convenience.
rereduced = sp.Matrix(Mredsymsym)

# Prepare for the test just by multiplication with some general vector of symbols.
vec = sp.Matrix(sp.symbols('A, B, C'))

# Multiply the \tilde{M}^D_{red} by the (A,B,C)^T vector.
M_vec = rereduced*vec

Check that the last (5th) row of $\bar{M}^D_{\text{red}}$ is the same as the next-to-last row (4th) multiplied by $\sin(\theta)^2$:

In [20]:
sp.simplify(M_vec[3]*b - M_vec[4])

0

Check that the next-to-last row (4th) of $\bar{M}^D_{\text{red}}$ is the same as the third row multiplied by $-\frac{\gamma^{rr}A_{rr}}{2\gamma^{\theta\theta}}$:

In [21]:
sp.simplify(M_vec[2]*gammaUU[0][0]/(2*gammaUU[1][1]) + M_vec[3])

0

Prepare the RHS from the equation given above. That is the expression:

 \begin{align}
 \begin{pmatrix} 
 \text{RHS}_{\Phi} \\
\text{RHS}_{K}\\
\text{RHS}_{A_{rr}} \\
\text{RHS}_{A_{\theta\theta}}\\
\text{RHS}_{A_{\phi\phi}}
 \end{pmatrix} = \begin{pmatrix}
    \mathcal{S}^{\Phi} \\
    \mathcal{S}^K \\
    \mathcal{S}^{A}_{rr} \\ 
    \mathcal{S}^{A}_{\theta\theta} \\
    \mathcal{S}^{A}_{\phi\phi} 
\end{pmatrix}
+ \frac{ A^{ij}A_{ij}}{\gamma^{22}}
\begin{pmatrix}
    \tilde{M}_{\Phi A} \\
    \tilde{M}_{K A}\\
    \tilde{M}_{AA \, rr} \\
    \tilde{M}_{AA \, \theta \theta}\\
    \tilde{M}_{AA \, \phi \phi}
\end{pmatrix}  \end{align}


In [22]:
AA = sp.sympify(0)
#  Tensors are diagonal here no need to loop all the zeros.
for i in range(DIM):
    AA += sGB.ADD[i][i]**2*gammaUU[i][i]**2

factor = AA/gammaUU[1][1]

# RHS_i vector
S_RHS = sp.Matrix(S) - factor*reduced[:, -1]

$\text{RHS}_{A_{\theta\theta}} \sin{\theta}^2 - \text{RHS}_{A_{\phi\phi}}$

In [24]:
sp.simplify(S_RHS[3]*b - S_RHS[4])

0

$\text{RHS}_{A_{rr}} \frac{\gamma^{rr}}{2\gamma^{\theta\theta}} + \text{RHS}_{A_{\theta\theta}}$

In [25]:
sp.simplify(S_RHS[2]*gammaUU[0][0]/(2*gammaUU[1][1]) + S_RHS[3])

0